### Imports

In [35]:
#Imports
import re
import numpy as np
import pandas as pd
import sklearn.linear_model as lm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from vectorization import hash_vectorizer

ImportError: cannot import name 'hash_vectorizer' from 'vectorization' (c:\Users\User\Desktop\DNA_CNT\Code\vectorization.py)

### Functions

In [8]:
#Functions

# One hot encoding ##############################

def onehote(sequence):
    mapping = {"A": 0, "C": 1, "G": 2, "T": 3}
    seq2 = [mapping[i] for i in sequence]
    return np.eye(4,dtype=int)[seq2]
#One hot encoding on a list
def seq_encoding(list):
    return np.array([onehote(i) for i in list])
#Dataframe column tolist().
def toList(dataframe,column_name):
    return dataframe[column_name].tolist()

#################################################

# One hot encoding on chirality #################

#Get integers from chirality string
def toInt(chirality_string):
    buff = re.split(r"[();]",chirality_string)
    return [int(buff[1]),int(buff[2])]

#Get the chirality from the list of chirality strings
def getChirality(chirality_list):
    return np.array([toInt(i) for i in chirality_list])

#Onehot encode the chirality number
def onehot_chirality(number):
    return np.eye(12,dtype=int)[number]
#Onehot encode the chirality
def extract_chirality(chirality):
    return np.array([onehot_chirality(i) for i in chirality])
#Apply onehot on the chirality list
def apply_chirality(chirality_list):
    return np.array([onehot_chirality(i) for i in chirality_list])

#################################################

# Misc ##########################################

#Flatten the array
def Flatten(list):
    return np.array([list[i].flatten() for i in range(len(list))])

#Make a tuple with index and value from a list
def make_tuple(list):
    return [(i+1,list[i]) for i in range(len(list))]
#Make a tuple from two lists
def make_tuple2(list1,list2):
    if(len(list1) != len(list2)):
        print("Error: Lists are not the same length")
        return None
    return [(list1[i],list2[i]) for i in range(len(list1))]
#################################################

### Data Import

In [14]:
# Load the data
set_raw = pd.read_csv('../Data/training_set.csv', low_memory=False)

# Preprocess the data
# Chirality
chirality = getChirality(set_raw['Chirality'])
chirality = pd.DataFrame(chirality, columns=['m','n'])

### Count Vectorizer

In [33]:
# Transform
# Sequences
seq = vectorization.vectorizer.transform(set_raw['Sequence'])
seq = pd.DataFrame(seq.toarray(), columns=vectorization.vectorizer.get_feature_names_out())
# Merge two dataframes
set = pd.concat([seq,chirality,set_raw['Label']],axis=1)
set.sample(5)

,c,cc,ccc,cct,ct,ctc,ctt,t,tc,tcc,tct,tt,ttc,ttt,m,n,Label
711,6,1,0,1,4,2,2,6,4,1,2,2,2,0,8,5,N
300,6,5,4,1,1,0,1,6,1,1,0,4,1,2,10,2,N
46,5,0,0,0,5,3,1,7,5,0,5,1,1,0,9,7,N
304,6,5,4,1,1,0,1,6,1,1,0,4,1,2,9,6,Y
853,8,4,0,3,3,2,1,4,3,3,0,1,1,0,10,3,N


In [34]:
vectorization.hash_vectorizer

AttributeError: module 'vectorization' has no attribute 'hash_vectorizer'

### Hash Vectorizer

In [29]:
# Transform
# Sequences
seq_hash = vectorization.hash_vectorizer.transform(set_raw['Sequence'])
seq_hash = pd.DataFrame(seq_hash.toarray())
# Remove columns with all zeros
seq_hash = seq_hash.loc[:, (seq_hash != 0).any(axis=0)]
seq_hash.head()

AttributeError: module 'vectorization' has no attribute 'hash_vectorizer'

### Model

In [22]:
def mymodel():
    model = lm.LogisticRegression(multi_class='multinomial', solver='lbfgs')
    return model

In [ ]:
#Train test split
X_train, X_test, Y_train, Y_test = train_test_split(test,
                                                    set_raw['Label'],
                                                    test_size=0.2,
                                                    random_state=42)
#Train the model
model = mymodel()
model.fit(X_train, Y_train)